###Step1. Make the TextCNN Model

In [ ]:
import torch.nn as nn #신경망 구축을 위한 데이터 구조나 layer 등의 라이브러리
import torch.optim as optim #파라미터 최적화 알고리즘 제공
import torch.nn.functional as F #활성화 함수, 손실 함수 등 신경망 제작에 쓰이는 함수 제공

In [ ]:
class TextCNN(nn.Module):
  
  '''
  vocab_built: Train 데이터로 생성한 단어장을 인자로 받음
  emb_dim: 임베딩 벡터의 크기
  dim_channel: feature map 이후 생성되는 채널의 수
  kernel_wins: filter의 크기를 리스트 형태로 받음
  num_class: output 클래스 개수
  '''
  def __init__(self, vocal_built, emb_dim, dim_channel, kernel_wins, num_class):

    super(TextCNN, self).__init__()

    self.embed = nn.Embedding(len(vocab_built), emb_dim) #임베딩 설정,
                                                           #vocab_size * embedding dimension
    self.embed.weight.data.copy_(vocab_built.vectors)      #Word2Vec로 학습한 embedding vector 값
    
    self.convs = nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
    
    self.relu = nn.ReLU() #activation map 생성
    self.dropout = nn.Dropout(0.4) #overfitting 방지, 즉 가중치 행렬에서 랜덤 40%에 0을 넣어 연산
    self.fc = nn.Linear(len(kernel_wins)*dim_channel, num_classes)    #score vector 생성
    
    
  def forward(self, x):
    emb_x = self.embed(x)
    emb_x = emb_x.unsqueeze(1) #batch*1*vocab size*embedding vector
                               #2차원의 텍스트 데이터를 모델에 이미지처럼 입력하기 위해 3차원 형태로 변환해야 함
    
    con_x = [self.relu(conv(emb_x)) for conv in self.convs]
    
    pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in convs]
    
    fc_x = torch.cat(pool_x, dim=1)
    fc_x = fc_x.squeeze(-1)
    fc_x = self.dropout(fc_x)
    
    logit = self.fc(fc_x)
    
    return logit       

###Step2. Make the train function

In [ ]:
def train(model, device, train_itr, optimizer):

  model.train()
  corrects, train_loss = 0.0, 0

  for batch in train_iter: #batch 사이즈별로 train
													#batch가 곧 dataset
    text, target = batch.text, batch.label #TEXT, LABEL; 


    text = torch.transpose(text, 0, 1) #전치행렬 적용
    target.data.sub_(1)
    text, target = text.to(device), target.to(device)

    ######################################

		optimizer.zero_grad() #optimizer의 gradient 초기화
    logit = model(text) #예측값
    loss = F.cross_entropy(logit, target) #loss 값
    loss.backward() #back propogation
    optimizer.step() #파라미터 별 할당된 gradient 값 이용해 파라미터 업데이트

    train_loss += loss.item() #미니 배치 loss 누적
    result = torch.max(logit, 1)[1] #행 기준으로 제일 큰 값의 인덱스 출력
    corrects += (result.view(target.size()).data == target.data).sum()

  train_loss /= len(train_iter.dataset)
  accuacy = 100.0*corrects / len(train_itr.dataset)

  return train_loss, accuracy